In [29]:
#conda install conda-forge::feature_engine

#!pip install feature_engine

   ---------------------------------------- 0.0/357.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/357.1 kB ? eta -:--:--
   ------ -------------------------------- 61.4/357.1 kB 825.8 kB/s eta 0:00:01
   -------------------------------------- - 348.2/357.1 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 357.1/357.1 kB 2.8 MB/s eta 0:00:00


In [185]:
# Import des différents packages.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
import copy
import pickle
from joblib import dump, load

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.feature_selection import VarianceThreshold

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

#from imblearn.over_sampling import RandomOverSampler, SMOTE
#from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression,Ridge,Lasso


In [101]:
from IPython.display import display_markdown
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [102]:
import os
root_folder_path = os.path.abspath('..')
data_processed_path = os.path.join(root_folder_path, "data", "processed")

In [133]:
df = pd.read_csv(os.path.join(data_processed_path, r"base_donnees_nettoyee_non_dupliquee.csv"), sep=",", index_col=0)

C:\Users\courio\AppData\Local\Temp\ipykernel_16272\1611470788.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_processed_path, r"base_donnees_nettoyee_non_dupliquee.csv"), sep=",", index_col=0)


In [134]:
df = df.drop(['DateOfCall','DateAndTimeMobilised', 'DateAndTimeArrived',"PlusCode_Description","DateOfCall_In_Ceconds"], axis=1)

In [136]:
# https://stackoverflow.com/questions/51102205/how-to-know-the-labels-assigned-by-astypecategory-cat-codes
df["IncidentGroup"] = df["IncidentGroup"].astype('category').cat.codes
df["StopCodeDescription"] = df["StopCodeDescription"].astype('category').cat.codes
df["PropertyCategory"] = df["PropertyCategory"].astype('category').cat.codes
df["AddressQualifier"] = df["AddressQualifier"].astype('category').cat.codes
#df["IncGeo_BoroughName"] = df["IncGeo_BoroughName"].astype('category').cat.codes
df["PerformanceReporting"] = df["PerformanceReporting"].map({'1': 1, '2': 2, 'Not Used': np.nan,})
df["PerformanceReporting"] = df["PerformanceReporting"].fillna(1)
df["PlusCode_Code"] = df["PlusCode_Code"].astype('category').cat.codes
df["PlusCode_Code"] = df["PlusCode_Code"].astype('category').cat.codes

In [138]:
df.isna().sum().sum()

0

In [139]:
df.head()

,CalYear,HourOfCall,IncidentGroup,StopCodeDescription,PropertyCategory,AddressQualifier,Postcode_district,IncGeo_BoroughName,Latitude,Longitude,...,PumpOrder,PlusCode_Code,DateOfCall_Dayofweek,DateOfCall_Quarter,DateOfCall_Month,DateOfCall_Dayofyear,DateOfCall_Dayofmonth,DateOfCall_Weekofyear,MinutesOfCall,FirstPumpArriving_AttendanceTime_InMinutes
0,2009,0,2,7,8,1,SW11,WANDSWORTH,51.475812,-0.148894,...,1,0,3,1,1,1,1,1,0,5.316667
1,2009,0,1,6,5,2,N9,ENFIELD,51.633342,-0.068488,...,1,0,3,1,1,1,1,1,3,5.133333
2,2009,0,1,6,5,6,UB10,HILLINGDON,51.533882,-0.448089,...,1,0,3,1,1,1,1,1,4,3.500000
3,2009,0,1,6,5,2,N7,ISLINGTON,51.551441,-0.111120,...,1,0,3,1,1,1,1,1,5,3.883333
4,2009,0,0,0,2,0,NW5,CAMDEN,51.551297,-0.134346,...,1,0,3,1,1,1,1,1,6,2.866667


In [140]:
numeric_features = [
    #'DateOfCall',
    #'DateAndTimeMobilised', 'DateAndTimeArrived',
    'CalYear', 'HourOfCall','Latitude','Longitude','PumpMinutesRounded',
    'DateOfCall_Dayofweek',
    'DateOfCall_Quarter',
    'DateOfCall_Month', 'DateOfCall_Dayofyear',
    'DateOfCall_Dayofmonth', 'DateOfCall_Weekofyear',
    #'DateOfCall_In_Ceconds',
    'MinutesOfCall',
    'FirstPumpArriving_AttendanceTime_InMinutes'
    ]

categorical_features = [item for item in df.columns if item not in numeric_features]

numeric_features.remove('FirstPumpArriving_AttendanceTime_InMinutes')
#categorical_features.remove('FirstPumpArriving_AttendanceTime_InMinutes')

In [141]:
numeric_features

['CalYear',
 'HourOfCall',
 'Latitude',
 'Longitude',
 'PumpMinutesRounded',
 'DateOfCall_Dayofweek',
 'DateOfCall_Quarter',
 'DateOfCall_Month',
 'DateOfCall_Dayofyear',
 'DateOfCall_Dayofmonth',
 'DateOfCall_Weekofyear',
 'MinutesOfCall']

In [142]:
categorical_features

['IncidentGroup',
 'StopCodeDescription',
 'PropertyCategory',
 'AddressQualifier',
 'Postcode_district',
 'IncGeo_BoroughName',
 'NumStationsWithPumpsAttending',
 'NumPumpsAttending',
 'PumpCount',
 'NumCalls',
 'Resource_Code',
 'PerformanceReporting',
 'DeployedFromStation_Name',
 'DeployedFromLocation',
 'PumpOrder',
 'PlusCode_Code']

In [143]:
print(len(df.columns))
print(len(numeric_features))
print(len(categorical_features))
print(len(numeric_features)+len(categorical_features))

29
12
16
28


In [144]:
y = df['FirstPumpArriving_AttendanceTime_InMinutes']
X = df.drop(['FirstPumpArriving_AttendanceTime_InMinutes'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [145]:
countFrequencyEncoder = CountFrequencyEncoder(encoding_method="frequency", variables=["Postcode_district", "Resource_Code","DeployedFromStation_Name","IncGeo_BoroughName"])
countFrequencyEncoder.fit(X_train)
X_train = countFrequencyEncoder.transform(X_train)
X_test = countFrequencyEncoder.transform(X_test)

g:\Anaconda\envs\Jan24_cds_Pompiers\Lib\site-packages\feature_engine\encoding\base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) Postcode_district, Resource_Code, DeployedFromStation_Name.
  warnings.warn(


In [148]:
X_train.isna().sum().sum()

0

In [124]:
X_train.head()

,CalYear,HourOfCall,IncidentGroup,StopCodeDescription,PropertyCategory,AddressQualifier,Postcode_district,IncGeo_BoroughName,Latitude,Longitude,...,DeployedFromLocation,PumpOrder,PlusCode_Code,DateOfCall_Dayofweek,DateOfCall_Quarter,DateOfCall_Month,DateOfCall_Dayofyear,DateOfCall_Dayofmonth,DateOfCall_Weekofyear,MinutesOfCall
787490,2017,7,1,5,3,0,0.005675,0.033002,51.507005,-0.446148,...,1,2,0,4,1,1,6,6,1,30
525633,2014,8,0,0,2,0,0.005232,0.033818,51.512411,-0.297346,...,1,1,0,2,1,1,8,8,2,5
1212505,2021,20,0,2,2,0,0.003654,0.021572,51.527641,0.193392,...,1,1,0,5,2,5,149,29,21,23
1056130,2019,23,2,7,2,10,0.004680,0.044272,51.483296,-0.099676,...,1,1,0,5,3,9,271,28,39,4
778252,2016,15,0,2,3,0,0.000597,0.011041,51.514367,-0.106328,...,1,1,0,2,4,11,335,30,48,31


In [155]:
# https://stackoverflow.com/questions/65418378/valueerror-a-given-column-is-not-a-column-of-the-dataframe
def build_pipeline(algo_ml, impute_num = SimpleImputer(strategy="median"), impute_var=SimpleImputer(strategy="constant", fill_value="Unknown"), scaler = StandardScaler()):
  # Pipeline de transformation des variables numériques
  numeric_transformer = make_pipeline(
    (impute_num), 
    (scaler)
    )
  
  # Pipeline de transformation des variables catégorielles
  categorical_transformer = make_pipeline(
    #(impute_var),
    (OneHotEncoder(handle_unknown="ignore"))
    )
  
  # Combinaison des 2 étapes en un seul objet
  preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
    )
  
  # Pipeline final de Machine Learning
  model = Pipeline(steps=[('preprocessing', preprocessor), 
                        ('linear', algo_ml)])
  
  return model

In [85]:
build_pipeline(algo_ml=LinearRegression())

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CalYear', 'HourOfCall',
                                                   'Latitude', 'Longitude',
                                                   'PumpMinutesRounded',
                                                   'DateOfCall_In_Ceconds',
                                                   'DateOfCall_Dayofweek',
                                                   'DateOfCall_Quarter',
                                                   'DateOfCall_Month',
                                                   'DateOfCall_Dayofy...
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['IncidentGroup',
                                                   'StopCodeDescription',
                                                   'PropertyCategory',
                                                   'AddressQualifier',
                                                   'Postcode_district',
                                                   'IncGeo_BoroughName',
                                                   'NumStationsWithPumpsAttending',
                                                   'NumPumpsAttending',
                                                   'PumpCount', 'NumCalls',
                                                   'Resource_Code',
                                                   'PerformanceReporting',
                                                   'DeployedFromStation_Name',
                                                   'DeployedFromLocation',
                                                   'PumpOrder',
                                                   'PlusCode_Code'])])),
                ('linear', LinearRegression())])

In [156]:
model = build_pipeline(algo_ml = LinearRegression())
model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CalYear', 'HourOfCall',
                                                   'Latitude', 'Longitude',
                                                   'PumpMinutesRounded',
                                                   'DateOfCall_Dayofweek',
                                                   'DateOfCall_Quarter',
                                                   'DateOfCall_Month',
                                                   'DateOfCall_Dayofyear',
                                                   'DateOfCall_Dayofmo...
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['IncidentGroup',
                                                   'StopCodeDescription',
                                                   'PropertyCategory',
                                                   'AddressQualifier',
                                                   'Postcode_district',
                                                   'IncGeo_BoroughName',
                                                   'NumStationsWithPumpsAttending',
                                                   'NumPumpsAttending',
                                                   'PumpCount', 'NumCalls',
                                                   'Resource_Code',
                                                   'PerformanceReporting',
                                                   'DeployedFromStation_Name',
                                                   'DeployedFromLocation',
                                                   'PumpOrder',
                                                   'PlusCode_Code'])])),
                ('linear', LinearRegression())])

In [157]:
model.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CalYear', 'HourOfCall',
                                                   'Latitude', 'Longitude',
                                                   'PumpMinutesRounded',
                                                   'DateOfCall_Dayofweek',
                                                   'DateOfCall_Quarter',
                                                   'DateOfCall_Month',
                                                   'DateOfCall_Dayofyear',
                                                   'DateOfCall_Dayofmo...
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['IncidentGroup',
                                                   'StopCodeDescription',
                                                   'PropertyCategory',
                                                   'AddressQualifier',
                                                   'Postcode_district',
                                                   'IncGeo_BoroughName',
                                                   'NumStationsWithPumpsAttending',
                                                   'NumPumpsAttending',
                                                   'PumpCount', 'NumCalls',
                                                   'Resource_Code',
                                                   'PerformanceReporting',
                                                   'DeployedFromStation_Name',
                                                   'DeployedFromLocation',
                                                   'PumpOrder',
                                                   'PlusCode_Code'])])),
                ('linear', LinearRegression())])

In [159]:
print("Score Train = ", model.score(X_train,y_train))
print("Score Test= ", model.score(X_test,y_test))

Score Train =  0.15793572771517472
Score Test=  0.155857546828016


In [181]:
# https://www.dataquest.io/blog/understanding-regression-error-metrics/
# https://medium.com/codenx/machine-learning-using-scikit-learn-sklearn-evaluating-regression-model-using-metrics-0414107a7e22
y_pred = model.predict(X_test)
# The coefficients
#print("Coefficients: \n", model.coef_)
# The mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_pred))
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Mean absolute error: 1.16
Mean squared error: 2.26
Coefficient of determination: 0.16


In [178]:
# https://stackoverflow.com/questions/41659535/valueerror-x-and-y-must-be-the-same-size
# Plot outputs
#plt.scatter(X_train[:,0], y_train, color="black")
#plt.scatter(X_test[:, 0], y_test, color="black")
#plt.scatter(X_test.iloc[:,22].values, y_test, color="black")

#plt.plot(X_test.iloc[:,22].values, y_pred, color="blue", linewidth=3)
#plt.xticks(())
#plt.yticks(())
#plt.show()

In [182]:
X_train

,CalYear,HourOfCall,IncidentGroup,StopCodeDescription,PropertyCategory,AddressQualifier,Postcode_district,IncGeo_BoroughName,Latitude,Longitude,...,DeployedFromLocation,PumpOrder,PlusCode_Code,DateOfCall_Dayofweek,DateOfCall_Quarter,DateOfCall_Month,DateOfCall_Dayofyear,DateOfCall_Dayofmonth,DateOfCall_Weekofyear,MinutesOfCall
787490,2017,7,1,5,3,0,0.005675,0.033002,51.507005,-0.446148,...,1,2,0,4,1,1,6,6,1,30
525633,2014,8,0,0,2,0,0.005232,0.033818,51.512411,-0.297346,...,1,1,0,2,1,1,8,8,2,5
1212505,2021,20,0,2,2,0,0.003654,0.021572,51.527641,0.193392,...,1,1,0,5,2,5,149,29,21,23
1056130,2019,23,2,7,2,10,0.004680,0.044272,51.483296,-0.099676,...,1,1,0,5,3,9,271,28,39,4
778252,2016,15,0,2,3,0,0.000597,0.011041,51.514367,-0.106328,...,1,1,0,2,4,11,335,30,48,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,2011,4,0,2,8,2,0.002839,0.030005,51.481076,0.102105,...,1,1,0,4,2,4,105,15,15,27
1414427,2023,19,0,3,6,0,0.002934,0.033818,51.502140,-0.364139,...,1,1,0,4,1,3,62,3,9,24
131932,2010,22,0,3,2,0,0.019983,0.038716,51.352415,-0.016054,...,1,1,0,5,1,2,37,6,5,40
671157,2015,13,0,0,4,10,0.006311,0.031492,51.515080,-0.214884,...,1,1,0,5,3,8,241,29,35,43


In [73]:
X_train[categorical_features].head()

,IncidentGroup,StopCodeDescription,PropertyCategory,AddressQualifier,Postcode_district,IncGeo_BoroughName,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,NumCalls,Resource_Code,PerformanceReporting,DeployedFromStation_Name,DeployedFromLocation,PumpOrder,PlusCode_Code
787490,1,5,3,0,0.005675,0.033002,2.0,2.0,3,1.0,0.005629,2.0,0.005727,1,2,0
525633,0,0,2,0,0.005232,0.033818,2.0,2.0,2,1.0,0.007513,1.0,0.009400,1,1,0
1212505,0,2,2,0,0.003654,0.021572,1.0,1.0,1,2.0,0.002577,1.0,0.002645,1,1,0
1056130,2,7,2,10,0.004680,0.044272,1.0,1.0,1,1.0,0.010288,1.0,0.017467,1,1,0
778252,0,2,3,0,0.000597,0.011041,1.0,1.0,1,1.0,0.010336,1.0,0.010567,1,1,0


In [184]:
model_ridge = build_pipeline(algo_ml = Ridge(alpha=.5))
model_ridge.fit(X_train,y_train)
print("Score Train = ", model_ridge.score(X_train,y_train))
print("Score Test= ", model_ridge.score(X_test,y_test))

Score Train =  0.1579356739654032
Score Test=  0.1558500172683147


In [187]:
model_lasso = build_pipeline(algo_ml = Lasso(alpha=.5))
model_lasso.fit(X_train,y_train)
print("Score Train = ", model_lasso.score(X_train,y_train))
print("Score Test= ", model_lasso.score(X_test,y_test))

Score Train =  0.0
Score Test=  -2.7563521921125655e-05


In [189]:
print("Avec ces scores très très bas, avant de continuer avec les CV,tuning etc... , on va revenir sur le preprocessing.")

Avec ces scores très très bas, avant de continuer avec les CV,tuning etc... , on va revenir sur le preprocessing.


In [190]:
# https://www.simplilearn.com/tutorials/scikit-learn-tutorial/sklearn-regression-models
# https://www.kaggle.com/discussions/general/309745